In [1]:
# Update each first day of the month with the previous month forecast

# Countryrisk

In [17]:
import wbdata
import pandas as pd
import pandas_profiling
import datetime
import quandl
%config IPCompleter.greedy=True
import pandas_datareader
import urllib3


# 1- Extracting and Cleaning the Data from IMF, WB and BIS

## Country selection


In [18]:
### fecthing country names from IMF database to get all ISO codes
countrycodes = pd.read_csv('coucodes.csv', delimiter = ';', encoding = "ISO-8859-1")
countrycodes = countrycodes[["IMF Name", "ISO Code", "ISO2 Code"]]
countrycodes.reset_index(level=0, inplace=True)
countrycodes = countrycodes.rename(columns={'IMF Name': 'country'})

countries = ["Brazil",  "Mexico", "India",  "Russia", "Uruguay",
             "Korea", "Thailand", "Costa Rica", "Colombia", "Paraguay",
             "Chile", "South Africa", "Taiwan", "Turkey", "Ukraine", "Nigeria", "Indonesia",
             "Bangladesh", "Philippines", "Pakistan", "Egypt", "Ethiopia", "Vietnam", 
             "Myanmar", "Algeria", "Sudan", "Uganda", "Morocco",  "Malaysia",
             "Afghanistan", "Ghana", "Laos", "Singapore", "Poland",
             "Israel", "Czech Republic", "Romania", "Hungary", "Kazakhstan", "Kenya", "Angola",
            "Dominican Republic", "Sri Lanka", "Guatemala", "Bulgaria", "Tanzania", "Belaurus",
            "Croatia",  "Democratic Republic of the Congo",
            "Azerbaijan", "Côte d'Ivoire", "Argentina", "Iran", "Iraq", "Venezuela"]

countrycodes2 = countrycodes.copy()
countrycodes2 = countrycodes2[["country", "ISO2 Code"]]
countrycodes2 = countrycodes2.dropna()

#Argentina, Australia,  Venezuela e Iraq are not working - Monthly
#
countrycodes = countrycodes[countrycodes["country"].isin(countries)]

In [19]:
from datetime import datetime
from itertools import product

today = datetime.today()
datem = datetime(today.year, today.month, 1)

year = pd.date_range('1995-01-01', end = str(today.month) + "-" + str(today.year), freq='MS').strftime('%Y-%m')


base = pd.DataFrame(data=list(product(year, countrycodes["ISO2 Code"])), columns=['year','ISO2 Code'])
base = base.sort_values(by=['ISO2 Code', "year"]).dropna()


In [20]:
todaydate = pd.date_range(start = str(today.month) + "-" + str(today.year), end = str(today.month) + "-" + str(today.year), freq='MS').strftime('%Y-%m')
base = base[base["year"] != base["year"].max()]

## WB


In [21]:
#downloading data from worldbank

from pandas_datareader import wb

worldbank_data = pandas_datareader.wb.download(indicator = ["DSTKMKTXD"], country = countrycodes["ISO Code"], start = 2010, end = 2022, freq = "M")
worldbank_data2 = pandas_datareader.wb.download(indicator =["DSTKMKTXD"], country = countrycodes["ISO Code"], start = 1999, end = 2009, freq = "M")
worldbank_data3 = pandas_datareader.wb.download(indicator = ["DSTKMKTXD"], country = countrycodes["ISO Code"], start = 1990, end = 1998, freq = "M")

worldbank_data = pd.concat([worldbank_data, worldbank_data2])
worldbank_data = pd.concat([worldbank_data, worldbank_data3])

worldbank_data.reset_index(inplace=True)

worldbank_data_conc = pd.merge(worldbank_data, countrycodes, on="country")
worldbank_data_conc["year"] = pd.to_datetime(worldbank_data_conc["year"], format='%YM%m')
worldbank_data_conc.drop(["country", "ISO Code", "index"],1, inplace = True)


## IMF

In [43]:
#Run R script to import imf data - R package makes the task much easier

import subprocess
retcode = subprocess.call(['C:/Program Files/R/R-3.5.1/bin/Rscript.exe', '--vanilla', 
                           'C:/Users/vitor/OneDrive/Documentos/databowl/teste.R'])

In [107]:
stack = pd.read_csv("C:/Users/vitor/OneDrive/Documentos/databowl/teste2.csv")
stack =stack.rename(columns = {'x_gdp':'PCTOT', "iso2c": "ISO2 Code", "year_month": "year"})
stack.drop("Unnamed: 0", inplace = True, axis = 1)
stack["year"] = pd.to_datetime(stack["year"], format='%Y-%m')
stack['FM2_XDC'] = "NaN"

In [40]:
stack2 = pd.read_csv("backup.csv")
stack2 =stack2.rename(columns = {'x_gdp':'PCTOT', "iso2c": "ISO2 Code", "year_month": "year"})
stack2.drop("Unnamed: 0", inplace = True, axis = 1)
stack2["year"] = pd.to_datetime(stack2["year"], format='%Y-%m')


In [60]:
stack = stack[stack["year"] == str(today.year) + "-" + str(today.month-1) + "-" + "1"]

In [63]:
staques = [stack2, stack]
stack = pd.concat(staques, sort = False)

,ISO2 Code,year,FM2_XDC,RAXGFX_USD,LUR_PT,TXG_FOB_USD,TMG_CIF_USD,FDSAOP_XDC,AIP_IX,ENDE_XDC_USD_RATE,...,FISR_PA,FDSF_XDC,FDSB_XDC,RAXG_USD,PCPI_IX,FASMB_XDC,PCTOT,NGDP_XDC,BGS_BP6_USD,BCAXF_BP6_USD
0,AE,1990-01-01,NaN,4095.8,NaN,NaN,958.594388,NaN,NaN,3.671000,...,NaN,NaN,NaN,4393.156770,NaN,NaN,69.382545,NaN,NaN,NaN
1,AE,1990-02-01,NaN,4236.3,NaN,NaN,846.635794,NaN,NaN,3.671000,...,NaN,NaN,NaN,4536.835942,NaN,NaN,68.176208,NaN,NaN,NaN
2,AE,1990-03-01,NaN,4367.6,NaN,NaN,1014.982294,NaN,NaN,3.671000,...,NaN,NaN,NaN,4656.293186,NaN,NaN,66.420372,NaN,NaN,NaN
3,AE,1990-04-01,NaN,4253.0,NaN,NaN,947.425770,NaN,NaN,3.671000,...,NaN,NaN,NaN,4542.044896,NaN,NaN,64.369209,NaN,NaN,NaN
4,AE,1990-05-01,NaN,4246.0,NaN,NaN,1113.320621,NaN,NaN,3.671000,...,NaN,NaN,NaN,4533.552972,NaN,NaN,64.168175,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79525,WS,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79887,YE,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80249,ZA,2020-01-01,NaN,45260.0,NaN,NaN,NaN,NaN,NaN,14.905995,...,NaN,NaN,NaN,48252.589030,NaN,267165.643448,NaN,NaN,NaN,NaN
80611,ZM,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,228.074879,NaN,NaN,NaN,NaN,NaN


## OECD

In [106]:
stack2

,ISO2 Code,year,FM2_XDC,RAXGFX_USD,LUR_PT,TXG_FOB_USD,TMG_CIF_USD,FDSAOP_XDC,AIP_IX,ENDE_XDC_USD_RATE,...,FISR_PA,FDSF_XDC,FDSB_XDC,RAXG_USD,PCPI_IX,FASMB_XDC,PCTOT,NGDP_XDC,BGS_BP6_USD,BCAXF_BP6_USD
0,AE,1990-01-01,NaN,4095.800000,NaN,NaN,958.594388,NaN,NaN,3.6710,...,NaN,NaN,NaN,4393.156770,NaN,NaN,69.382545,NaN,NaN,NaN
1,AE,1990-02-01,NaN,4236.300000,NaN,NaN,846.635794,NaN,NaN,3.6710,...,NaN,NaN,NaN,4536.835942,NaN,NaN,68.176208,NaN,NaN,NaN
2,AE,1990-03-01,NaN,4367.600000,NaN,NaN,1014.982294,NaN,NaN,3.6710,...,NaN,NaN,NaN,4656.293186,NaN,NaN,66.420372,NaN,NaN,NaN
3,AE,1990-04-01,NaN,4253.000000,NaN,NaN,947.425770,NaN,NaN,3.6710,...,NaN,NaN,NaN,4542.044896,NaN,NaN,64.369209,NaN,NaN,NaN
4,AE,1990-05-01,NaN,4246.000000,NaN,NaN,1113.320621,NaN,NaN,3.6710,...,NaN,NaN,NaN,4533.552972,NaN,NaN,64.168175,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80331,ZW,2019-07-01,NaN,131.688295,NaN,NaN,NaN,5545.475996,NaN,9.1856,...,NaN,-20016.403381,17076.036256,137.619125,NaN,3769.321375,99.090439,NaN,NaN,NaN
80332,ZW,2019-08-01,NaN,174.810935,NaN,NaN,NaN,6099.787635,NaN,10.7139,...,NaN,-23628.960936,19694.065282,179.504155,NaN,5747.625567,NaN,NaN,NaN,NaN
80333,ZW,2019-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80334,ZW,2019-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
full = pd.merge(stack, worldbank_data_conc, left_on = ["ISO2 Code", "year"], right_on = ["ISO2 Code", "year"], how = "left")


## Couprisk

In [109]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re

url = "https://oefdatascience.github.io/REIGN.github.io/menu/reign_current.html"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
for link in soup.find_all(attrs={'href': re.compile("http")}):
    links.append(link.get('href'))


sub = "https://cdn.rawgit.com/OEFDataScience/REIGN.github.io/gh-pages/data_sets/REIGN"


link = [word for word in links if word.startswith(sub)]

import pandas as pd
couprisk = pd.read_csv(link[0])
couprisk = couprisk[["country", "year", "month", "couprisk", "ccode"]]
couprisk["month"] = couprisk["month"].astype(int)
couprisk["month"] = couprisk.month.map("{:02}".format)
couprisk["year"] = couprisk["year"].round().astype(int).astype(str) + "-" + couprisk["month"].astype(str)+"-" + "01"



In [110]:
cowcode = pd.read_csv("coucodes2.csv", sep = ";")
cowcode = cowcode.drop_duplicates()

#couprisk = couprisk[couprisk["country"].isin(countries)]
couprisk = pd.merge(couprisk, cowcode, left_on = "ccode", right_on = "cowcode2")

couprisk = couprisk.drop(["country", "ccode", "cowcode2"], 1)
couprisk = couprisk.drop_duplicates(["year", "ccdcodelet"])
couprisk = couprisk.rename(columns={'ccdcodelet': 'ISO Code'})
couprisk = pd.merge(couprisk, countrycodes, on = "ISO Code")
couprisk = couprisk.drop(["index", "ISO Code", "country"], 1)


In [111]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re

url = "https://oefdatascience.github.io/REIGN.github.io/menu/reign_current.html"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
for link in soup.find_all(attrs={'href': re.compile("http")}):
    links.append(link.get('href'))


sub = "https://cdn.rawgit.com/OEFDataScience/REIGN.github.io/gh-pages/data_sets/electionlist"


link = [word for word in links if word.startswith(sub)]

import pandas as pd
election = pd.read_csv(link[0])
election = election[["country", "elec_year", "ccode", "elec_month"]]
election["elec_month"] = election["elec_month"].fillna(1).astype(int)
election["elec_month"] = election.elec_month.map("{:02}".format)
election["elec_year"] = election["elec_year"].round().astype(int).astype(str) + "-" + election["elec_month"].astype(str)+"-" + "01"


In [112]:
election = pd.merge(election, cowcode, left_on = "ccode", right_on = "cowcode2")

election = election.drop(["country", "ccode", "cowcode2"], 1)
#election = election.drop_duplicates(["year", "ccdcodelet"])
election = election.rename(columns={'ccdcodelet': 'ISO Code', "elec_year" : "year"})
election = pd.merge(election, countrycodes, on = "ISO Code")
election = election.drop(["index", "ISO Code", "country", "elec_month"], 1)
election["election"] = 1
election = election.drop_duplicates()

In [113]:
couprisk["year"] = pd.to_datetime(couprisk["year"], errors = "coerce")
election["year"] = pd.to_datetime(election["year"], errors = "coerce")


full = pd.merge(full, couprisk, on = ["ISO2 Code", "year"], how = "left")
full = pd.merge(full, election, on = ["ISO2 Code", "year"], how = "left")

full["election"] = full["election"].fillna(0)
full = full[full["ISO2 Code"] != "RO"]
full = full[full["ISO2 Code"] != "BH"]
full = full[full.year.notnull()]

In [114]:
GGXWDG_NGDP = pd.read_csv("GGXWDG_NGDP.csv", error_bad_lines=False, sep = ";")
GGXWDG_NGDP = pd.melt(GGXWDG_NGDP, id_vars=['country'])
GGXWDG_NGDP = pd.merge(GGXWDG_NGDP, countrycodes2, on = "country", how = "left")
GGXWDG_NGDP = GGXWDG_NGDP.dropna()
GGXWDG_NGDP["variable"] = pd.to_datetime(GGXWDG_NGDP["variable"], errors = "coerce")
GGXWDG_NGDP = GGXWDG_NGDP.rename(columns = {"value": "GGXWDG_NGDP", "variable" : "year"})

GGXCNL_NGDP = pd.read_csv("GGXCNL_NGDP.csv", error_bad_lines=False, sep = ";")
GGXCNL_NGDP = pd.melt(GGXCNL_NGDP, id_vars=['country'])
GGXCNL_NGDP = pd.merge(GGXCNL_NGDP, countrycodes2, on = "country", how = "left")
GGXCNL_NGDP = GGXCNL_NGDP.dropna()
GGXCNL_NGDP["variable"] = pd.to_datetime(GGXCNL_NGDP["variable"], errors = "coerce")
GGXCNL_NGDP = GGXCNL_NGDP.rename(columns = {"value": "GGXCNL_NGDP", "variable" : "year"})

full = pd.merge(full, GGXWDG_NGDP, on = ["ISO2 Code", "year"], how = "left")
full =pd.merge(full,  GGXCNL_NGDP, on = ["ISO2 Code", "year"], how = "left")

full = full.drop(["country_x", "country_y"],1)

In [115]:
import yfinance as yf

msft = yf.Ticker("^VIX")
vix = msft.history(period="max")


In [116]:
import quandl
quandl.ApiConfig.api_key = "e2KZ8SFfstro3DH2_zvj"

oil = quandl.get("ODA/POILWTI_USD")
oil.index = oil.index + pd.offsets.MonthBegin(0)
oil.reset_index(inplace = True)
oil = oil.rename(columns = {"Date": "year", "Value":"oil"})
full = pd.merge(full, oil, on = "year", how = "left")

In [117]:
full

,ISO2 Code,year,RAXGFX_USD,LUR_PT,TXG_FOB_USD,TMG_CIF_USD,FDSAOP_XDC,AIP_IX,ENDE_XDC_USD_RATE,FMB_XDC,...,BGS_BP6_USD,BCAXF_BP6_USD,FM2_XDC,DSTKMKTXD,month,couprisk,election,GGXWDG_NGDP,GGXCNL_NGDP,oil
0,AE,1990-01-01,4095.800000,NaN,NaN,958.594388,NaN,NaN,3.6710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,21.130
1,AE,1990-02-01,4236.300000,NaN,NaN,846.635794,NaN,NaN,3.6710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22.960
2,AE,1990-03-01,4367.600000,NaN,NaN,1014.982294,NaN,NaN,3.6710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,22.120
3,AE,1990-04-01,4253.000000,NaN,NaN,947.425770,NaN,NaN,3.6710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,20.410
4,AE,1990-05-01,4246.000000,NaN,NaN,1113.320621,NaN,NaN,3.6710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,18.430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75444,ZW,2019-09-01,94.037809,NaN,NaN,NaN,6875.497283,NaN,15.1979,23551.729789,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,54.839
75445,ZW,2019-10-01,118.281238,NaN,NaN,NaN,7820.402437,NaN,15.6740,28930.062344,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,56.857
75446,ZW,2019-11-01,112.523420,NaN,NaN,NaN,9712.660415,NaN,16.2575,31820.551599,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,53.979
75447,ZW,2019-12-01,NaN,NaN,NaN,NaN,NaN,NaN,16.7734,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,57.110


In [118]:
#vix
vix.sort_index(inplace=True)
vix2 = vix.copy()
vix2.index = pd.to_datetime(vix2.index) + pd.offsets.MonthBegin(0)
vix2 = vix2["Open"].to_frame()
vix2 = vix2.loc[~vix2.index.duplicated(keep='first')]
vix2.reset_index(inplace = True)
vix2 = vix2.rename(columns = {"Date": "year", "Open":"vix"})

full = pd.merge(full, vix2, on = "year", how = "left")


In [119]:
currency = full.sort_values(by=['year']).copy()


In [120]:
full = full[full["ISO2 Code"].isin(countrycodes["ISO2 Code"])]

# 2 - Creating variables and cleaning the Data

In [121]:


#to numeric
cols = full.columns.drop(["ISO2 Code", "year"])

full[cols] = full[cols].apply(pd.to_numeric, errors='coerce')

In [122]:
import numpy as np
teste = full.copy()
teste.set_index(["year", "ISO2 Code"], inplace = True)
teste[cols] = teste[cols].apply(pd.to_numeric, errors='coerce')
teste = teste.notnull().astype('int')
teste.reset_index(inplace = True)
teste.set_index("year", inplace = True)
teste.replace(0, np.nan, inplace=True)
teste.reset_index(inplace = True)

In [123]:
#teste = pd.merge(countrycodes2, teste, how = "left", on = "ISO2 Code")
#teste.set_index("year", inplace = True)
#teste.groupby("country").idxmax().to_csv("first.csv")

In [124]:
#teste = teste.iloc[::-1]
#teste.groupby("country").idxmax().to_csv("last.csv")

In [125]:
#Counting missing values by country
#full2 = full
#full2 = pd.merge(countrycodes2, full2, how = "right", on = "ISO2 Code")
#g = full2.groupby('country')
#g.count().rsub(g.size(), axis=0).to_csv("missings.csv")

In [126]:
full

,ISO2 Code,year,RAXGFX_USD,LUR_PT,TXG_FOB_USD,TMG_CIF_USD,FDSAOP_XDC,AIP_IX,ENDE_XDC_USD_RATE,FMB_XDC,...,BCAXF_BP6_USD,FM2_XDC,DSTKMKTXD,month,couprisk,election,GGXWDG_NGDP,GGXCNL_NGDP,oil,vix
361,AF,1990-01-01,237.69,NaN,NaN,NaN,NaN,NaN,42.822780,NaN,...,NaN,NaN,NaN,1.0,0.007426,0.0,NaN,NaN,21.130,NaN
362,AF,1990-02-01,236.53,NaN,NaN,NaN,NaN,NaN,42.822780,NaN,...,NaN,NaN,NaN,2.0,0.008280,0.0,NaN,NaN,22.960,17.24
363,AF,1990-03-01,221.15,NaN,NaN,NaN,NaN,NaN,42.822780,NaN,...,NaN,NaN,NaN,3.0,0.006203,0.0,NaN,NaN,22.120,24.32
364,AF,1990-04-01,232.30,NaN,NaN,NaN,NaN,NaN,42.822780,NaN,...,NaN,NaN,NaN,4.0,0.009857,0.0,NaN,NaN,20.410,21.34
365,AF,1990-05-01,239.44,NaN,NaN,NaN,NaN,NaN,42.822780,NaN,...,NaN,NaN,NaN,5.0,0.008715,0.0,NaN,NaN,18.430,22.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74722,ZA,2019-09-01,45908.00,NaN,NaN,NaN,3.361741e+06,NaN,15.208940,3.765423e+06,...,NaN,NaN,100.334804,9.0,0.000836,0.0,NaN,NaN,54.839,17.69
74723,ZA,2019-10-01,45448.00,NaN,NaN,NaN,3.377365e+06,NaN,15.160060,3.776070e+06,...,NaN,NaN,NaN,10.0,0.000893,0.0,NaN,NaN,56.857,20.96
74724,ZA,2019-11-01,46032.00,NaN,NaN,NaN,3.391187e+06,NaN,14.651907,3.803032e+06,...,NaN,NaN,NaN,11.0,0.000904,0.0,NaN,NaN,53.979,18.75
74725,ZA,2019-12-01,45904.00,NaN,NaN,NaN,3.388652e+06,NaN,14.025541,3.763779e+06,...,NaN,NaN,NaN,12.0,0.000911,0.0,NaN,NaN,57.110,12.68


In [127]:
#to numeric
cols = full.columns.drop(["ISO2 Code", "year"])

full[cols] = full[cols].apply(pd.to_numeric, errors='coerce')



In [128]:
#creation
full["broad_reserves_ratio"] = full["FDSB_XDC"]/(full["RAXG_USD"]*full["ENDE_XDC_USD_RATE"]).diff(1)
full["broad_base_ratio"] = full["FDSB_XDC"]/full["FASMB_XDC"].diff(1)
full["foreignassets_reserves_ratio"] = full["FDSF_XDC"]/(full["RAXG_USD"]*full["ENDE_XDC_USD_RATE"]).diff(1)

full["inflation"] = full.groupby('ISO2 Code', sort=False).PCPI_IX.apply(
     lambda x: x.pct_change(12))

full["BCAXF_BP6_USD"] = full["BCAXF_BP6_USD"]*full["ENDE_XDC_USD_RATE"]/full["NGDP_XDC"]


full["current_account"] = full.groupby('ISO2 Code', sort=False).BCAXF_BP6_USD.apply(
     lambda x: x.pct_change(12)) 

full["moneymarket_rate"] = full.groupby('ISO2 Code', sort=False).FIMM_PA.apply(
     lambda x: x.pct_change(12))

full["reserves"] = full["RAXG_USD"].diff(1)
full["reserves_12p"] = full.groupby('ISO2 Code', sort=False).RAXG_USD.apply(
     lambda x: x.pct_change(12))

full["stocks"] = full.groupby('ISO2 Code', sort=False).DSTKMKTXD.apply(
     lambda x: x.pct_change(12))

full["commodities_index"] =  full.groupby('ISO2 Code', sort=False).PCTOT.apply(
     lambda x: x.pct_change(12))  

full["exports"] = full.groupby('ISO2 Code', sort=False).TXG_FOB_USD.apply(
     lambda x: x.pct_change(12))

full["imports"] = full.groupby('ISO2 Code', sort=False).TMG_CIF_USD.apply(
     lambda x: x.pct_change(12))

full["imports_fx"] = (full["TMG_CIF_USD"]/full["RAXG_USD"]).diff(1)

full["exchange_change"] = full.groupby('ISO2 Code', sort=False).ENDE_XDC_USD_RATE.apply(
     lambda x: x.pct_change(1)).shift(1)

full["exchange_change_12m"] = full.groupby('ISO2 Code', sort=False).ENDE_XDC_USD_RATE.apply(
     lambda x: x.pct_change(12))

full["net_debt"] = full.groupby('ISO2 Code', sort=False).GGXWDG_NGDP.apply(
     lambda x: x.pct_change(12))

full["lending_borrowing"] = full.groupby('ISO2 Code', sort=False).GGXCNL_NGDP.apply(
     lambda x: x.pct_change(12))

full["lending_rate"] = full.groupby('ISO2 Code', sort=False).FILR_PA.apply(
     lambda x: x.pct_change(12))

full["M2"] = full.groupby('ISO2 Code', sort=False).FM2_XDC.apply(
     lambda x: x.pct_change(12))

full["claims_private"] = full.groupby('ISO2 Code', sort=False).FDSAOP_XDC.apply(
     lambda x: x.pct_change(12)).diff(1)

full["vix"] = full.groupby('ISO2 Code', sort=False).vix.apply(
     lambda x: x.pct_change(12))

full["oil"] = full.groupby('ISO2 Code', sort=False).oil.apply(
     lambda x: x.pct_change(12))


full["lag_1"] =full.groupby("ISO2 Code")["election"].shift(-1).fillna(0)
full["lag_2"] =full.groupby("ISO2 Code")["election"].shift(-2).fillna(0)
full["lag_3"] =full.groupby("ISO2 Code")["election"].shift(-3).fillna(0)
full["lag_4"] =full.groupby("ISO2 Code")["election"].shift(-4).fillna(0)
full["lag_5"] =full.groupby("ISO2 Code")["election"].shift(-5).fillna(0)
full["lag_6"] =full.groupby("ISO2 Code")["election"].shift(-6).fillna(0)
full['election_p'] =  full["lag_1"] + full["lag_2"] + full["lag_3"] + full["lag_4"] + full["lag_5"] + full["lag_6"]
full['election_p'] = np.where(full['election_p'] > 0, 1, 0)

full["lag_1"] =full.groupby("ISO2 Code")["election"].shift(1).fillna(0)
full["lag_2"] =full.groupby("ISO2 Code")["election"].shift(2).fillna(0)
full["lag_3"] =full.groupby("ISO2 Code")["election"].shift(3).fillna(0)
full["lag_4"] =full.groupby("ISO2 Code")["election"].shift(4).fillna(0)
full["lag_5"] =full.groupby("ISO2 Code")["election"].shift(5).fillna(0)
full["lag_6"] =full.groupby("ISO2 Code")["election"].shift(6).fillna(0)
full['election'] =  full["lag_1"] + full["lag_2"] + full["lag_3"] + full["lag_4"] + full["lag_5"] + full["lag_6"]
full['election'] = np.where(full['election'] > 0, 1, 0)


#full["exchange_change"] = full.groupby('ISO2 Code', sort=False).ENDE_XDC_USD_RATE.apply(
     #lambda x: x.pct_change(12))

full.drop(["FASMB_XDC", "FDSB_XDC", "FDSF_XDC", "FIMM_PA",  "PCPI_IX", "RAXG_USD", "PCTOT",
           "BCAXF_BP6_USD", "DSTKMKTXD", "FILR_PA", "GGXCNL_NGDP", "GGXWDG_NGDP", "TXG_FOB_USD", "TMG_CIF_USD",
          "LUR_PT", "FDSAOP_XDC", "AIP_IX", "EREER_IX", "BGS_BP6_USD", "NGDP_XDC", "FM2_XDC", "RAXGFX_USD",
          "FMB_XDC", "FISR_PA", "lag_1", "lag_2", "lag_3", "lag_4", 'lag_5', "lag_6"],1, inplace = True)                                                         

In [129]:
full.set_index(["year", "ISO2 Code"], inplace = True)

#full = full.groupby(level=1).ffill()
full = full.replace([np.inf, -np.inf], np.nan)


In [130]:
full.exchange_change = full.exchange_change.fillna(0)


In [131]:
full

,,ENDE_XDC_USD_RATE,month,couprisk,election,oil,vix,broad_reserves_ratio,broad_base_ratio,foreignassets_reserves_ratio,inflation,...,imports,imports_fx,exchange_change,exchange_change_12m,net_debt,lending_borrowing,lending_rate,M2,claims_private,election_p
year,ISO2 Code,,,,,,,,,,,,,,,,,,,,,
1990-01-01,AF,42.822780,1.0,0.007426,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0
1990-02-01,AF,42.822780,2.0,0.008280,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0
1990-03-01,AF,42.822780,3.0,0.006203,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0
1990-04-01,AF,42.822780,4.0,0.009857,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0
1990-05-01,AF,42.822780,5.0,0.008715,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-01,ZA,15.208940,9.0,0.000836,1,-0.193414,0.303611,49.628311,1245.699510,11.981717,0.041436,...,-0.233213,NaN,0.070631,0.072279,0.056437,0.409091,0.00000,NaN,0.002061,0
2019-10-01,ZA,15.160060,10.0,0.000893,1,-0.189910,0.596344,-427.580676,-1454.137196,-104.136421,0.036664,...,-0.306041,NaN,-0.002169,0.025322,0.056437,0.409091,0.00000,NaN,0.005713,0
2019-11-01,ZA,14.651907,11.0,0.000904,1,-0.237057,0.503609,-233.820941,278.332603,-56.217220,0.035682,...,-0.262094,NaN,-0.003214,0.064630,0.056437,0.409091,-0.02439,NaN,-0.004779,0


In [132]:
#Dropping columns used to create the new variables
#Scaling

#full["ISO"] = full.index.get_level_values(1)
#full = pd.get_dummies(full, columns=['ISO'], prefix = ['Country'])

# Definition of dependent variable

In [133]:
import numpy as np
full_fill = full
#FIRST DEFINITION
full_fill["dummy"] = np.where(full_fill['exchange_change']  >0.2, 1, 0) #20 percent variation

#SECOND DEFINITION
#full_fill["threshold"] = full_fill.groupby("ISO2 Code")["exchange_change"].transform("mean") + (full_fill.groupby("ISO2 Code")["exchange_change"].transform("std"))
#full_fill["dummy"] = np.where(full_fill['exchange_change'] > full_fill["threshold"], 1, 0) # > 2 stdev threshold

#THIRD DEFINITION
#full_fill["dummy"] = np.where(full_fill['exchange_change_12m'].shift(-6) >0.2, 1, 0) #50 percent variation in the next 12 months


crise = full_fill

crise["lag_1"] =crise.groupby("ISO2 Code")["dummy"].shift(-1).fillna(0)
crise["lag_2"] =crise.groupby("ISO2 Code")["dummy"].shift(-2).fillna(0)
crise["lag_3"] =crise.groupby("ISO2 Code")["dummy"].shift(-3).fillna(0)
crise["lag_4"] =crise.groupby("ISO2 Code")["dummy"].shift(-4).fillna(0)
crise["lag_5"] =crise.groupby("ISO2 Code")["dummy"].shift(-5).fillna(0)
crise["lag_6"] =crise.groupby("ISO2 Code")["dummy"].shift(-6).fillna(0)
crise["lag_7"] =crise.groupby("ISO2 Code")["dummy"].shift(-7).fillna(0)
crise["lag_8"] =crise.groupby("ISO2 Code")["dummy"].shift(-8).fillna(0)
crise["lag_9"] =crise.groupby("ISO2 Code")["dummy"].shift(-9).fillna(0)
crise["lag_10"] =crise.groupby("ISO2 Code")["dummy"].shift(-10).fillna(0)
crise["lag_11"] =crise.groupby("ISO2 Code")["dummy"].shift(-11).fillna(0)
crise["lag_12"] =crise.groupby("ISO2 Code")["dummy"].shift(-12).fillna(0)
crise["lag_13"] =crise.groupby("ISO2 Code")["dummy"].shift(-13).fillna(0)
crise["lag_14"] =crise.groupby("ISO2 Code")["dummy"].shift(-14).fillna(0)
crise["lag_15"] =crise.groupby("ISO2 Code")["dummy"].shift(-15).fillna(0)
crise["lag_16"] =crise.groupby("ISO2 Code")["dummy"].shift(-16).fillna(0)
crise["lag_17"] =crise.groupby("ISO2 Code")["dummy"].shift(-17).fillna(0)
crise["lag_18"] =crise.groupby("ISO2 Code")["dummy"].shift(-18).fillna(0)
crise["lag_19"] =crise.groupby("ISO2 Code")["dummy"].shift(-19).fillna(0)
crise["lag_20"] =crise.groupby("ISO2 Code")["dummy"].shift(-20).fillna(0)
crise["lag_21"] =crise.groupby("ISO2 Code")["dummy"].shift(-21).fillna(0)
crise["lag_22"] =crise.groupby("ISO2 Code")["dummy"].shift(-22).fillna(0)
crise["lag_23"] =crise.groupby("ISO2 Code")["dummy"].shift(-23).fillna(0)
crise["lag_24"] =crise.groupby("ISO2 Code")["dummy"].shift(-24).fillna(0)
#crise["yt-1"] =crise.groupby("ISO2 Code")["dummy"].shift(0)
#crise["yt2-1"] = crise.groupby("ISO2 Code")["exchange_change"].shift(1)
#crise["yt2-2"] = crise.groupby("ISO2 Code")["exchange_change"].shift(2)
#crise["yt2-3"] = crise.groupby("ISO2 Code")["exchange_change"].shift(3)

crise["dummy"].fillna(0, inplace=True)


In [134]:
crise['indicator'] =  crise["lag_1"] + crise["lag_2"] + crise["lag_3"] + crise["lag_4"] + crise["lag_5"] + crise["lag_6"] + crise["lag_7"] + crise["lag_8"] + crise["lag_9"] + crise["lag_10"] + crise["lag_11"] +crise["lag_12"]
crise['indicator2'] = np.where(crise['indicator'] > 0, 1, 0)

#crise['indicator2'] = crise["dummy"] #Third Definition

#bs2["lag_1"] +
#crise['indicator'] = crise["lag_1"] +crise["lag_2"]  + crise["lag_3"] + crise["lag_4"] + crise["lag_5"] +crise["lag_6"]
#crise['indicator2'] = np.where(crise['indicator'] > 0, 1, 0)

#crise['indicator'] =  crise["lag_1"] + crise["lag_2"] + crise["lag_3"] + crise["lag_4"] + crise["lag_5"] + crise["lag_6"] + crise["lag_7"] + crise["lag_8"] + crise["lag_9"] + crise["lag_10"] + crise["lag_11"] +crise["lag_12"] + crise["lag_13"] + crise["lag_14"] + crise["lag_15"] + crise["lag_16"] + crise["lag_17"] + crise["lag_18"] + crise["lag_19"] + crise["lag_20"] + crise["lag_21"] + crise["lag_22"] + crise["lag_23"] +crise["lag_24"]
#crise['indicator2'] = np.where(crise['indicator'] > 0, 1, 0)

In [135]:
full_fill = crise
full_fill.reset_index(inplace = True)
full_fill.set_index("year", inplace = True)

In [136]:
### Preparing machine learning pipeline 

from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier 
from xgboost import XGBRegressor
from sklearn.neural_network import MLPClassifier

In [137]:
full_fill.set_index("ISO2 Code", append = True, inplace = True)
columns = full_fill.columns
columns = columns.drop(["lag_1", "lag_2", "lag_3", "lag_4", "lag_5", "lag_6", "lag_7", "lag_8", "lag_9", "lag_10", "lag_11", "lag_12", "lag_13", "lag_14", "lag_15", "lag_16", "lag_17", "lag_18", "lag_19", "lag_20", "lag_21", "lag_22", "lag_23", "lag_24", "indicator"])

# XGBoost conservative

In [138]:
full_fill2 = full_fill.query('year > "1990-12-01"')

#full_fill2 = full_fill2.fillna(full_fill2.mean())
#full_fill3 = full_fill2
#full_fill2 =  full_fill2[full_fill2.columns.drop(list(full_fill2.filter(regex='Country')))]
columns2 = columns.drop(list(full_fill.filter(regex='Country')))
#columns2 = ["inflation", "reserves_gdp", "commodities_growth", "couprisk", "ISO2 Code"]
X =  full_fill2[columns2]#.groupby(level=1).diff(1)
#X = X.drop(["ISO2 Code"],1)
X_lag = full_fill2[columns2].groupby(level=1).shift(1)
X_lag2 = full_fill2[columns2].groupby(level=1).shift(2)
X_lag3 = full_fill2[columns2].groupby(level=1).shift(3)
X_lag4 = full_fill2[columns2].groupby(level=1).diff(12).shift(4)
X_lag5 = full_fill2[columns2].groupby(level=1).diff(12).shift(5)

X_level = full_fill2[columns2]

X_lag = X_lag.add_suffix('_lag')
X_lag2 = X_lag2.add_suffix('_lag2')
X_lag3 = X_lag3.add_suffix('_lag3')
X_lag4 = X_lag4.add_suffix('_lag4')
X_lag5 = X_lag5.add_suffix('_lag5')

X_level = X_level.add_suffix('_level')




Y = full_fill2['indicator2']
#X = pd.concat((X, full_fill3.filter(regex='Country')), axis=1)
#X = pd.concat([X, X_lag], axis=1)
#X = pd.concat([X, X_lag2], axis=1)
#X = pd.concat([X, X_lag3], axis=1)
#X = pd.concat([X, X_lag4], axis=1)
#X = pd.concat([X, X_lag5], axis=1)
#X = pd.concat([X, X_level], axis=1)

#X = X.fillna(X.mean())

#X = X.groupby('ISO2 Code_level').transform(lambda x: (x - x.mean()) / x.std())

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(X)
#X = pd.DataFrame(scaler.transform(X), columns=X.columns, index = X.index)

X = X.sort_index(level = 0) #For CV purpose
#X = X.drop(["ENDE_XDC_USD_RATE"], 1)
Y = Y.sort_index(level = 0) #For CV purpose

#X = X.fillna(X.mean())

X_final = X.drop(["indicator2", "month"],1)
#cols_to_shift = X_final.columns.drop("dummy")
X_final = X_final#.groupby(level = 1).shift()


X_train = X_final.query('year < "2009-12-31"')
X_test = X_final.query('year > "2009-12-31"')

Y_train = Y.to_frame().query('year < "2009-12-31"').iloc[:,0]
Y_test = Y.to_frame().query('year > "2009-12-31"').iloc[:,0]





In [139]:
import joblib
clf1 = joblib.load('regularmodel.joblib') 
clf2 = joblib.load('undermodel.joblib') 

In [140]:
lista = full_fill2.index.get_level_values(1).unique()


y_pred = clf1.predict(X_final)
y_prob = clf1.predict_proba(X_final)[:,1]
#y_pred =  (y_prob >= 0.2).astype('int')

#confusion_matrix(Y_test, y_pred)

X3 = X_final.copy()
X3["threshold"] = 0.5

X3["missing_ratio"] = X3.isnull().sum(axis=1) / len(X3.columns)


X3["pred"] = y_prob.copy()
#X3["pred"] = (X3["pred"] - X3["pred"].min())/(X3["pred"].max())
X3["precrisis"] = Y.copy()



X3.reset_index(inplace=True)


X3.set_index("year")

import plotly.express as px
import plotly.graph_objects as go
from plotly import tools

for iso in lista:

    X2 = X3[X3["ISO2 Code"] == iso]
    
    X2["ENDE_XDC_USD_RATE"] = X2["ENDE_XDC_USD_RATE"].ffill()
    X2["ENDE_XDC_USD_RATE"] = X2["ENDE_XDC_USD_RATE"].apply(pd.to_numeric, errors='coerce')
    X2["ENDE_XDC_USD_RATE"] = (X2["ENDE_XDC_USD_RATE"])/np.nanmax(X2["ENDE_XDC_USD_RATE"])
    #cycle, trend = sm.tsa.filters.hpfilter(X2["pred"], lamb=5)
    #X2["hp"] = trend
    #X2.plot(y= ["dummy","pred", "ENDE_XDC_USD_RATE"], linewidth=5, figsize=(15,15))






    X2.reset_index(inplace = True)
    X2 = X2.merge(countrycodes[["country", "ISO2 Code"]], on = "ISO2 Code")

    fig1 = go.Figure()



    fig1 = go.Bar(x=X2["year"],
                    y=X2["dummy"],
                    name='<b>Crisis Period<b>',
                    marker_color='#3CA3CB',
                    width = 2678400000
                       )


    fig2 = go.Bar(x=X2["year"],
                    y=X2["precrisis"],
                    name='<b>Pre-Crisis Periods<b>',
                    marker_color='#CECAC9',
                    marker_line_color = '#CECAC9',
                    opacity = 0.4,
                    width = 2678400000
                        )

    fig3 = go.Scatter(
                    x=X2["year"],
                    y=X2['pred'],
                    name="<b>Crisis Probability (Model Estimate)<b>",
                    line_color='#225A75',
                    opacity=1,
                    line=dict(color='deepskyblue', width=4
                                  ))

    fig4 = go.Scatter(
                    x=X2["year"],
                    y=X2["missing_ratio"],
                    name="<b>Share of Missing Data<b>",
                    line_color='#CECAC9',
                    opacity=1,
                    line=dict(color='black', width=4
                                  ))

    fig5  = go.Scatter(
                    x=X2["year"],
                    y=X2["ENDE_XDC_USD_RATE"],
                    name="<b>Exchange Rate per USD<b>",
                    line_color='#42A5BF',
                    opacity=0.6,
                    line=dict(color='firebrick', width=4))


    fig = tools.make_subplots(rows=3, cols=1, row_heights = [0.425, 0.425, 0.15], subplot_titles=('<b>Exchange Rate per USD<b>', 
                          '<b>Crisis Probability (Model Estimate)<b>', '<b>Share of Missing Data<b>'))

    fig.append_trace(fig1, 2, 1)
    fig.append_trace(fig2, 2, 1)
    fig.append_trace(fig3, 2, 1)
    fig.append_trace(fig4, 3, 1)
    fig.append_trace(fig5, 1, 1)


    fig['layout'].update(legend=dict(orientation='h',xanchor='center',x=0.5, font = dict(size = 15)), height=800, width=1300, plot_bgcolor='#ffffff', font = dict(family="Roboto", color = "#225A75", size = 20),
                         title =  dict(text = (X2['country'][1]), font =dict(family="Roboto",size=40, color = '#225A75')))

    fig.update_layout(
    font=dict(
        family="Roboto",
        color="#225A75"
    ))

    fig.update_layout(
    images=[dict(
        source="https://miro.medium.com/max/843/1*2mzh1JVx5IgZbSG7tVdI1Q.png",
        xref="paper", yref="paper",
        x=1, y=1.07,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )]
    )

    fig.update_yaxes(range=[0, 1], row=2, col=1)
    fig.update_yaxes(range=[0, 1], row=3, col=1)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black',  mirror='ticks')
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='#CECAC9')
    fig.update_xaxes(tickfont=dict(size=15, color='#786f6d'))
    fig.update_yaxes(tickfont=dict(size=15, color='#786f6d'))

    fig.write_image(iso + "stacked.png")


C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [141]:
X3["Forecast date"] = pd.to_datetime('today')
X3[["year", "ISO2 Code", "pred", "Forecast date"]]
today = datetime.today().strftime('%d-%m-%Y')
X3.to_csv(f'{today}XGB.csv')

## Undersampling

In [142]:
full_fill2 = full_fill.query('year > "1990-01-01"')
#full_fill2 = full_fill2.fillna(full_fill2.mean())
#full_fill3 = full_fill2
#full_fill2 =  full_fill2[full_fill2.columns.drop(list(full_fill2.filter(regex='Country')))]
columns2 = columns.drop(list(full_fill.filter(regex='Country')))
#columns2 = ["inflation", "reserves_gdp", "commodities_growth", "couprisk", "ISO2 Code"]
X =  full_fill2[columns2]#.groupby(level=1).diff(1)
#X = X.drop(["ISO2 Code"],1)
X_lag = full_fill2[columns2].groupby(level=1).shift(1)
X_lag2 = full_fill2[columns2].groupby(level=1).shift(2)
X_lag3 = full_fill2[columns2].groupby(level=1).shift(3)
X_lag4 = full_fill2[columns2].groupby(level=1).diff(12).shift(4)
X_lag5 = full_fill2[columns2].groupby(level=1).diff(12).shift(5)

X_level = full_fill2[columns2]

X_lag = X_lag.add_suffix('_lag')
X_lag2 = X_lag2.add_suffix('_lag2')
X_lag3 = X_lag3.add_suffix('_lag3')
X_lag4 = X_lag4.add_suffix('_lag4')
X_lag5 = X_lag5.add_suffix('_lag5')

X_level = X_level.add_suffix('_level')




Y = full_fill2['indicator2']
#X = pd.concat((X, full_fill3.filter(regex='Country')), axis=1)
#X = pd.concat([X, X_lag], axis=1)
#X = pd.concat([X, X_lag2], axis=1)
#X = pd.concat([X, X_lag3], axis=1)
#X = pd.concat([X, X_lag4], axis=1)
#X = pd.concat([X, X_lag5], axis=1)
#X = pd.concat([X, X_level], axis=1)

#X = X.fillna(X.mean())

#X = X.groupby('ISO2 Code_level').transform(lambda x: (x - x.mean()) / x.std())

#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#scaler.fit(X)
#X = pd.DataFrame(scaler.transform(X), columns=X.columns, index = X.index)

X = X.sort_index(level = 0) #For CV purpose
#X = X.drop(["ENDE_XDC_USD_RATE"], 1)
Y = Y.sort_index(level = 0) #For CV purpose

#X = X.fillna(X.mean())

X_final = X.drop(["indicator2", "month"],1)
#cols_to_shift = X_final.columns.drop("dummy")
X_final = X_final#.groupby(level = 1).shift()


X_train = X_final.query('year < "2009-12-31"')
X_test = X_final.query('year > "2009-12-31"')

Y_train = Y.to_frame().query('year < "2009-12-31"').iloc[:,0]
Y_test = Y.to_frame().query('year > "2009-12-31"').iloc[:,0]


train = pd.merge(X_train, Y_train, left_index=True, right_index=True)
count_class_0, count_class_1 = train.indicator2.value_counts()

df_class_0 = train[train['indicator2'] == 0]
df_class_1 = train[train['indicator2'] == 1]

#count_class_0, count_class_1 = train..value_counts()

df_class_0_under = df_class_0.sample(count_class_1, random_state = 1984)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

X_train = df_test_under.drop("indicator2", 1)
X_train.sort_index(inplace = True)
#X_train.dropna(inplace = True)
Y_train = df_test_under["indicator2"]
Y_train.sort_index(inplace = True)




In [143]:
lista = full_fill2.index.get_level_values(1).unique()


y_pred = clf2.predict(X_final)
y_prob = clf2.predict_proba(X_final)[:,1]
#y_pred =  (y_prob >= 0.2).astype('int')

#confusion_matrix(Y_test, y_pred)

X3 = X_final.copy()
X3["threshold"] = 0.5

X3["missing_ratio"] = X3.isnull().sum(axis=1) / len(X3.columns)


X3["pred"] = y_prob
#X3["pred"] = (X3["pred"] - X3["pred"].min())/(X3["pred"].max())
X3["precrisis"] = Y



X3.reset_index(inplace=True)


X3.set_index("year")

import plotly.express as px
import plotly.graph_objects as go
from plotly import tools

for iso in lista:

    X2 = X3[X3["ISO2 Code"] == iso]
    
    X2["ENDE_XDC_USD_RATE"] = X2["ENDE_XDC_USD_RATE"].ffill()
    X2["ENDE_XDC_USD_RATE"] = X2["ENDE_XDC_USD_RATE"].apply(pd.to_numeric, errors='coerce')
    X2["ENDE_XDC_USD_RATE"] = (X2["ENDE_XDC_USD_RATE"])/np.nanmax(X2["ENDE_XDC_USD_RATE"])
    #cycle, trend = sm.tsa.filters.hpfilter(X2["pred"], lamb=5)
    #X2["hp"] = trend
    #X2.plot(y= ["dummy","pred", "ENDE_XDC_USD_RATE"], linewidth=5, figsize=(15,15))






    X2.reset_index(inplace = True)
    X2 = X2.merge(countrycodes[["country", "ISO2 Code"]], on = "ISO2 Code")

    fig1 = go.Figure()



    fig1 = go.Bar(x=X2["year"],
                    y=X2["dummy"],
                    name='<b>Crisis Period<b>',
                    marker_color='#3CA3CB',
                    width = 2678400000
                       )


    fig2 = go.Bar(x=X2["year"],
                    y=X2["precrisis"],
                    name='<b>Pre-Crisis Periods<b>',
                    marker_color='#CECAC9',
                    marker_line_color = '#CECAC9',
                    opacity = 0.4,
                    width = 2678400000
                        )

    fig3 = go.Scatter(
                    x=X2["year"],
                    y=X2['pred'],
                    name="<b>Crisis Probability (Model Estimate)<b>",
                    line_color='#225A75',
                    opacity=1,
                    line=dict(color='deepskyblue', width=4
                                  ))

    fig4 = go.Scatter(
                    x=X2["year"],
                    y=X2["missing_ratio"],
                    name="<b>Share of Missing Data<b>",
                    line_color='#CECAC9',
                    opacity=1,
                    line=dict(color='black', width=4
                                  ))

    fig5  = go.Scatter(
                    x=X2["year"],
                    y=X2["ENDE_XDC_USD_RATE"],
                    name="<b>Exchange Rate per USD<b>",
                    line_color='#42A5BF',
                    opacity=0.6,
                    line=dict(color='firebrick', width=4))


    fig = tools.make_subplots(rows=3, cols=1, row_heights = [0.425, 0.425, 0.15], subplot_titles=('<b>Exchange Rate per USD<b>', 
                          '<b>Crisis Probability (Model Estimate)<b>', '<b>Share of Missing Data<b>'))

    fig.append_trace(fig1, 2, 1)
    fig.append_trace(fig2, 2, 1)
    fig.append_trace(fig3, 2, 1)
    fig.append_trace(fig4, 3, 1)
    fig.append_trace(fig5, 1, 1)


    fig['layout'].update(legend=dict(orientation='h',xanchor='center',x=0.5, font = dict(size = 15)), height=800, width=1300, plot_bgcolor='#ffffff', font = dict(family="Roboto", color = "#225A75", size = 20),
                         title =  dict(text = (X2['country'][1]), font =dict(family="Roboto",size=40, color = '#225A75')))

    fig.update_layout(
    font=dict(
        family="Roboto",
        color="#225A75"
    ))

    fig.update_layout(
    images=[dict(
        source="https://miro.medium.com/max/843/1*2mzh1JVx5IgZbSG7tVdI1Q.png",
        xref="paper", yref="paper",
        x=1, y=1.07,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )]
    )

    fig.update_yaxes(range=[0, 1], row=2, col=1)
    fig.update_yaxes(range=[0, 1], row=3, col=1)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black',  mirror='ticks')
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='#CECAC9')
    fig.update_xaxes(tickfont=dict(size=15, color='#786f6d'))
    fig.update_yaxes(tickfont=dict(size=15, color='#786f6d'))

    fig.write_image(iso + "stacked_under.png")


C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vitor\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

In [144]:
X3["Forecast date"] = pd.to_datetime('today')
X3[["year", "ISO2 Code", "pred", "Forecast date"]]
today = datetime.today().strftime('%d-%m-%Y')
X3.to_csv(f'{today}XGBunder.csv')

In [146]:
stack.to_csv("backup.csv")